In [1]:
%load_ext autoreload
%autoreload 2

# Android Workloads Experiments

In [2]:
import logging
reload(logging)
log_fmt = '%(asctime)-9s %(levelname)-8s: %(message)s'
logging.basicConfig(format=log_fmt)

# Change to info once the notebook runs ok
logging.getLogger().setLevel(logging.INFO)

In [3]:
%matplotlib inline

import collections
import copy
import devlib
import json
import matplotlib.pyplot as plt
import os
import pexpect as pe
import pandas as pd
import sqlite3
import trappy

from env import TestEnv
from android import Screen, Workload
from trace import Trace
from time import sleep

# Globals Setup

Devlib requires the ANDROID_HOME environment variable configured to point to your local installation of the Android SDK. If you have not this variable configured in the shell used to start the notebook server, you need to run the next cell to define where your Android SDK is installed.

In [4]:
# Setup Androd SDK
os.environ['ANDROID_HOME'] = '/home/vagrant/lisa/tools/android-sdk-linux/'
adb = !echo $ANDROID_HOME/platform-tools/adb
adb = adb[0]
logging.info("ADB      : %s", adb)
fastboot = !echo $ANDROID_HOME/platform-tools/fastboot
fastboot = fastboot[0]
logging.info("FASTBOOT : %s", fastboot)

# Setup Catapult for Systrace usage
CATAPULT_HOME = "/home/vagrant/catapult"
logging.info("CATAPULT : %s", CATAPULT_HOME)

2016-09-28 17:55:38,568 INFO    : ADB      : /home/vagrant/lisa/tools/android-sdk-linux//platform-tools/adb
2016-09-28 17:55:38,577 INFO    : FASTBOOT : /home/vagrant/lisa/tools/android-sdk-linux//platform-tools/fastboot
2016-09-28 17:55:38,579 INFO    : CATAPULT : /home/vagrant/catapult


In [5]:
# Android device to target
DEVICE = '2185A639000468F1'

# Ensure ADB is running as root
!{adb} -s {DEVICE} root

adbd is already running as root


# Support Functions

This set of support functions will help us running the benchmark using different CPUFreq governors.

## CPUFreq setup

In [6]:
def set_performance():
    target.cpufreq.set_all_governors('performance')

def set_powersave():
    target.cpufreq.set_all_governors('powersave')

def set_interactive():
    logging.info('Set CPUFreq: Interactive')
    target.cpufreq.set_all_governors('interactive')

def set_sched():
    logging.info('Set CPUFreq: SchedFreq')
    target.cpufreq.set_all_governors('sched')

def set_schedutil():
    logging.info('Set CPUFreq: SchedUtil')
    target.cpufreq.set_all_governors('schedutil')
    
def set_ondemand():
    target.cpufreq.set_all_governors('ondemand')
    
    for cpu in target.list_online_cpus():
        tunables = target.cpufreq.get_governor_tunables(cpu)
        target.cpufreq.set_governor_tunables(
            cpu,
            'ondemand',
            **{'sampling_rate' : tunables['sampling_rate_min']}
        )
        
def set_min_freqs(min_freq, cpus=None):
    scaling_min_freq = '/sys/devices/system/cpu/cpu{}/cpufreq/scaling_min_freq'
    if not cpus:
        cpus = target.list_online_cpus()
    for cpu in cpus:
        attr = scaling_min_freq.format(cpu)
        target.execute('echo {} > {}'.format(min_freq, attr))

## System reboot support

In [7]:
from subprocess import Popen, PIPE
import select

def wait_for_logcat_idle(seconds=1):
    lines = 0
    
    # Clear logcat
    os.system('{} logcat -s {} -c'.format(adb, DEVICE));

    # Dump logcat output
    logcat_cmd = '{} -s {} logcat'.format(adb, DEVICE)
    logcat = Popen(logcat_cmd, shell=True, stdout=PIPE)
    logcat_poll = select.poll()
    logcat_poll.register(logcat.stdout, select.POLLIN)

    # Monitor logcat until it's idle for the specified number of [s]
    logging.info("Waiting for system to be almost idle")
    logging.info("   i.e. at least 10[s] of no logcat messages")
    while True: 
        poll_result = logcat_poll.poll(seconds * 1000)
        print '\rLogcat lines counting: {}'.format(lines),
        if not poll_result:
            break            
        lines = lines + 1
        line = logcat.stdout.readline(1024)


def reboot(conf_name):
    
    try:

        # Get image path from conf
        # If not specified we just reboot, this should ensure similar
        # conditions for each experiment
        image = confs[conf_name]['boot_img']
        boot_img = boot_images_folder + image
        
        logging.info('Rebooting into configuration: %s...', conf_name)

        logging.debug('Reboot bootloader...')
        !{adb} -s {DEVICE} reboot-bootloader
        sleep(6)

        logging.info('Flash boot image: %s', boot_img)
        !{fastboot} -s {DEVICE} flash boot {boot_img}
        
        logging.debug('Rebooting device...')
        !{fastboot} -s {DEVICE} reboot
        
        logging.debug('Waiting 60[s] for target to reboot...')
        sleep(60)
        
        rebooting = True
    except:
        # No bootimage specified, using current one but still reboot
        # to start the experiment on the same conditions
        logging.info('No image for configuration: %s...', conf_name)
        rebooting = False
        pass


    # Restart ADB in root mode
    !{adb} -s {DEVICE} root
    # Disable charge via USB
    !{adb} shell "echo 0 >/sys/class/power_supply/battery/charging_enabled"
    # Log current kernel version
    output = target.execute('uname -a')
    logging.info('Running with kernel:')
    logging.info('   %s', output)
    
    # Unlock device screen (assume no password required)
    target.execute('input keyevent 82')
    
    # Wait for the system to complete the boot
    if rebooting:
        wait_for_logcat_idle(10)

# Benchmarks

## IOPerformance Benchmark

In [8]:
from collections import namedtuple
import pandas as pd
import re
DD_METRICS_RE = re.compile(
    r'(?P<bytes>[0-9]+).* \((?P<mbytes>[0-9\.]+).* (?P<time_s>[0-9\.]+).* (?P<mbps>[0-9]+)')

IOPerf_Columns = ['tag', 'operation', 'MBytes', 'Time', 'MBps']
IOPerf = namedtuple('IOPerf', ','.join(IOPerf_Columns))

# INFILE = '/dev/block/platform/soc/624000.ufshc/by-name/userdata'
# Define SSD device name for your target device
INFILE = '/dev/block/platform/soc/f723d000.dwmmc0/by-name/userdata'

def ioperf_operation(tag, block_size, block_count, operation, iterations=1):
    read = True if operation.upper() == 'READ' else False
    operation = 'Read' if read else 'Write'
    data = []

# NOTE: we use a statically linked version of dd which support the oflag=dsync
#       option, this measn that dd returns when data have been flashed on disk
#       Without that option, the stats returned by dd for write operations
#       are not usable to build a statistic.

    DD_CMD = target.get_installed('dd')

    infile = INFILE
    outfile = '/data/local/tmp/dd_out.file'
    if read:
        cmd = '{} if={} of=/dev/null bs={} count={} 2>&1'\
                    .format(DD_CMD, infile, block_size, block_count)
    else:
        cmd = '{} if=/dev/zero of={} bs={} count={} oflag=dsync 2>&1'\
                    .format(DD_CMD, outfile, block_size, block_count)

    # Add CPUs pinning as required by conf
    try:
        cpus_mask = confs[tag]['cpus']
        taskset = target.get_installed('taskset')
        if not taskset:
            raise RuntimeError('Taskset tool not installed, target reconfiguration required')
        cmd = '{} {} {}'.format(taskset, cpus_mask, cmd)
        logging.info('Using CPUs mask: %s', cpus_mask)
    except KeyError:
        # No CPUs mask pinning required
        pass

    # Set minimum frequency required by conf
    try:
        min_freq = confs[tag]['min_freq']
        logging.info('Using MIN frequency: %d', min_freq)
    except KeyError:
        # No MIN frequency required
        min_freq = 0
        pass
    set_min_freqs(min_freq)

    for i in range(iterations):
        if read:
            # Disable caches
            target.execute('sync')
            target.execute('echo 3 > /proc/sys/vm/drop_caches')    

        output = target.execute(cmd)
        for line in output.splitlines():
            match = DD_METRICS_RE.search(line)
            if not match:
                continue
            ioperf = IOPerf(tag, operation,
                            float(match.group('mbytes')),
                            float(match.group('time_s')),
                            int(match.group('mbps')))
            logging.info('%3d - %s transfer speed: %d MB/sec',
                         i+1, operation, ioperf.MBps)
            data.append(ioperf)

    # Restore min frequency if changed for the experiment
    if min_freq:
        set_min_freqs(0)

    return data

def ioperf_run(tag, megabytes, iterations=1, operations='READ_WRITE'):
    data = []
    
    # Operations are in blocks of 1MB
    block_size = 1024*1024
    block_count = megabytes
    
    # Collect stats for each operation
    for operation in operations.split('_'):
        logging.info('Profiling %d %s using %dMB data...',
                     iterations, operation, megabytes)
        data.extend(ioperf_operation(tag, block_size, block_count, operation, iterations))

    # Build Padas DataFrame for collected data
    df = pd.DataFrame(data, columns=IOPerf_Columns)
    return df

# Experiments Execution Function

In [9]:
import subprocess as sp

#SYSTRACE_EVTS = 'gfx view freq idle sched eas stune walt'
#SYSTRACE_EVTS = 'gfx view freq idle sched sm power sync workq'
# SYSTRACE_EVTS = 'view freq idle sched'
SYSTRACE_EVTS = 'view freq idle sched eas stune walt tasks-wait'
SYSTRACE_BUFFSIZE_KB = " -b {} ".format(10 * 1024)
SYSTRACE_CMD  = CATAPULT_HOME                      \
                + "/systrace/systrace/systrace.py" \
                + SYSTRACE_BUFFSIZE_KB             \
                + SYSTRACE_EVTS                    \
                + " -e {} -o {} -t {}"

def experiment(wl, res_dir, conf_name, wload_name, collect=''):
    
    # Load workload params
    wload_kind = wload_name.split()[0]
    iterations = int(wload_name.split()[1])
    wload_tag = wload_name.split()[2]\
                .replace('https://youtu.be/', '')\
                .replace('?t=', '_')
            
    # Check for workload being available
    wload = Workload.get(te, wload_kind)
    if wload_kind not in ['IOPerf'] and not wload:
        return {}
    
    # Setup test results folder
    exp_dir = os.path.join(res_dir, conf_name, "{}_{}".format(wload_kind, wload_tag))
    os.system('mkdir -p {}'.format(exp_dir));

    # Configure governor
    confs[conf_name]['set']()
    
    # Configure screen to max brightness and no dimming
    Screen.set_brightness(target, percent=100)
    Screen.set_dim(target, auto=False)
    Screen.set_timeout(target, 60*60*10) # 10 hours should be enought for an experiment
    
    # Start the required tracing command
    if 'ftrace' in collect:
        # Start FTrace and Energy monitoring
        te.ftrace.start()
    elif 'systrace' in collect:
        # Get the systrace time
        match = re.search(r'systrace_([0-9]+)', collect)
        if match:
            systrace_time = match.group(1)
        else:
            logging.warning("Systrace time NOT defined, tracing for 10[s]")
            systrace_time = 10
        # Start systrace
        trace_file = os.path.join(exp_dir, 'trace.html')
        trace_cmd = SYSTRACE_CMD.format(DEVICE, trace_file, systrace_time)
        logging.info('SysTrace: %s', trace_cmd)
        systrace_output = pe.spawn(trace_cmd)
    
    ###########################
    # Run the required workload
    
    # Jankbench
    if 'Jankbench' in wload_name:
        db_file, nrg_report = wload.run(exp_dir, wload_tag, iterations, collect)

    # UiBench
    elif 'UiBench' in wload_name:
        test_name = wload_name.split()[2]
        duration_s = int(wload_name.split()[3])
        db_file, nrg_report = wload.run(exp_dir, test_name, duration_s, collect)

    # YouTube
    elif 'YouTube' in wload_name:
        video_url = wload_name.split()[2]
        video_duration_s = int(wload_name.split()[3])
        db_file, nrg_report = wload.run(exp_dir, video_url, video_duration_s, collect)
        
    # IO Latency (using dd)
    elif 'IOPerf' in wload_name:
        megabytes  = int(wload_name.split()[2])
        operations = wload_name.split()[3]
        
        # Run measurements
        if 'energy' in collect:
            te.emeter.reset()
        df = ioperf_run(conf_name, megabytes, iterations, operations)
        if 'energy' in collect:
            nrg_report = te.emeter.report(exp_dir)
        
        # Save measurements
        csv_file = os.path.join(exp_dir, 'data.csv')
        df.to_csv(csv_file)
        
        db_file = csv_file

    # RTApp based workloads
    elif 'RTApp' in wload_name:
        rtapp_kind = wload_name.replace('RTApp ', '')
        db_file, nrg_report = rtapp_run(rtapp_kind)

        
    ###########################
    
    # Stop the required trace command
    if 'ftrace' in collect:
        te.ftrace.stop()
        # Collect and keep track of the trace
        trace_file = os.path.join(exp_dir, 'trace.dat')
        te.ftrace.get_trace(trace_file)
        if 'functions' in my_conf['ftrace']:
            trace_stats = os.path.join(exp_dir, 'trace.stats')
            te.ftrace.get_stats(trace_stats)
    elif 'systrace' in collect:
        logging.info('Waiting systrace report [%s]...', trace_file)
        systrace_output.wait()

    # Reset screen brightness and auto dimming
    Screen.set_defaults(target, )
        
    # Dump platform descriptor
    te.platform_dump(exp_dir)

    # return all the experiment data
    if 'trace' in collect:
        return {
            'dir'             : exp_dir,
            'db_file'         : db_file,
            'nrg_report'      : nrg_report,
            'trace_file'      : trace_file,
        }
    else:
        return {
            'dir'             : exp_dir,
            'db_file'         : db_file,
            'nrg_report'      : nrg_report,
        }

In [10]:
def run_experiments(test_confs, verbose=False, pretend=False):

    # Make sure we have a list of configuraitons to test
    if not isinstance(test_confs, list):
        test_confs = [test_confs]

    # Intialize Workloads for this test environment
    wl = Workload(te)

    # Change to info once the notebook runs ok
    if verbose:
        logging.getLogger().setLevel(logging.DEBUG)
    else:
        logging.getLogger().setLevel(logging.INFO)

    # Unlock device screen (assume no password required)
    target.execute('input keyevent 82')

    # The set of results for each comparison test
    results = collections.defaultdict(dict)

    results = {}
    # Run the benchmark in all the configured configurations
    for conf_name in test_confs:
        
        results[conf_name] = {}

        # Flash and reboot with the required bootimage
        logging.info('\n\n')
        logging.info('========================')
        if not pretend:
            reboot(conf_name)

        # Setup data to be collected
        try:
            collect = confs[conf_name]['collect']
            logging.info("Enabling collection of: %s", collect)
        except:
            collect = ''

        # Enable energy collection only if an emeter has been configured
        if 'energy' in collect:
            if 'emeter' not in my_conf or not te.emeter:
                logging.warning('Disabling ENERGY collection')
                logging.info('Emeter not configured or not available')
                collect = collect.replace('energy', '')
            else:
                logging.debug('Enabling ENERGY collection')

        # Run each workload
        idx = 0
        for wload_name in wloads:
            
            # Skip workload if not enabled by the configuration
            try:
                enabled = False
                enabled_workloads = confs[conf_name]['wloads']
                for wload in enabled_workloads:
                    if wload in wload_name:
                        enabled = True
                        break
                if not enabled:
                    logging.debug('Workload [%s] disabled',
                                 wload_name)
                    continue
            except:
                # No workload filters defined, execute all workloads
                logging.debug('All workloads enabled')
                pass

            # Log test being executed
            idx = idx + 1
            wload_kind = wload_name.split()[0]
            logging.info('------------------------')
            logging.info('Test %d: %s in %s configuration',
                         idx, wload_kind.upper(), conf_name.upper())
            logging.info('        %s', wload_name)
            
            # Run commands only in "non pretend" mode
            if pretend:
                continue
            res = experiment(wl, te.res_dir, conf_name, wload_name, collect)
            results[conf_name][wload_name] = copy.deepcopy(res)

        # Save results only in "non pretend" mode
        if pretend:
            continue

        # Save collected results
        res_file = os.path.join(te.res_dir, conf_name, 'results.json')
        with open(res_file, 'w') as fh:
            json.dump(results[conf_name], fh, indent=4)
        
        return results

# Test Environment set up

In case more than one Android device are conencted to the host, you must specify the ID of the device you want to target in `my_target_conf`. Run `adb devices` on your host to get the ID.

In [11]:
# Setup target configuration
my_conf = {

    # Target platform and board
    "platform"    : 'android',
    "board"       : 'hikey',
    "device"      : DEVICE,

    "emeter" : {
        "instrument" : "acme",
        "conf" : {
            # Absolute path to the iio-capture binary on the host
            'iio-capture' : '/home/vagrant/iio-capture/iio-capture',
            # IP the BeagleBone Black
            'ip_address'     : '192.168.0.1',
        },
        "channel_map" : {
            "Device0" : 0, # iio:device0
            "Device1" : 1, # Hikey
        }
    },

    # Folder where all the results will be collected
    "results_dir" : "Android_WorkloadsExecution",

    # Define devlib modules to load
    "modules"     : [
        'cpufreq'       # enable CPUFreq support
    ],

    # FTrace events to collect for all the tests configuration which have
    # the "ftrace" flag enabled
    "ftrace"  : {
         "events" : [
            "sched_switch",
            "sched_wakeup",
            "sched_wakeup_new",
            "sched_wakeup_tracking",
            
            "sched_stat_wait",
            
            "sched_overutilized",
            "sched_contrib_scale_f",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_tune_config",
            "sched_tune_filter",
            "sched_tune_tasks_update",
            "sched_tune_boostgroup_update",
            "sched_boost_cpu",
            "sched_boost_task",
            "sched_energy_diff",
            "cpu_capacity",
            "cpu_frequency",
            "cpu_idle",
            "walt_update_task_ravg",
            "walt_update_history",
            "walt_migration_update_sum",
         ],
        "functions" : [
            "pick_next_task_fair",
            "select_task_rq_fair",
            "enqueue_task_fair",
            "update_curr_fair",
            "dequeue_task_fair",
        ],
        "buffsize" : 10 * 1024,
    },

    # Tools required by the experiments
    "tools"   : ['trace-cmd', 'dd', 'taskset', 'atrace'],
}

## Workload Configuration

In [12]:
# List of workloads to run, each workload consists of a workload name
# followed by a list of workload specific parameters
wloads = [

# YouTube workload:
# Params:
# - iterations: number of read/write operations to execute
# - URL:        link to the video to use (with optional start time)
# - duration:   playback time in [s]
#     'YouTube 1 https://youtu.be/XSGBVzeBUbk?t=45s   60',

# Jankbench workload:
# Params:
# - iterations: number of read/write operations to execute
# - id:         benchmakr to run
#     'Jankbench 1 list_view',
#     'Jankbench 1 image_list_view',
#     'Jankbench 1 shadow_grid',
#     'Jankbench 1 low_hitrate_text',
#     'Jankbench 1 high_hitrate_text',
#     'Jankbench 1 edit_text',
    
    # Multi iterations
#     'Jankbench 3 list_view',
#     'Jankbench 3 image_list_view',
#     'Jankbench 3 shadow_grid',
#     'Jankbench 3 low_hitrate_text',
#     'Jankbench 3 high_hitrate_text',
#     'Jankbench 3 edit_text',

# UiBench workload:
# Params:
# - test_name:  The name of the test to start
# - duration:   playback time in [s]
     'UiBench 1 TrivialAnimation 10',
    
# IO Performance workload (based on dd copies)
# Params:
# - iterations: number of read/write operations to execute
# - megabytes:  amout of data to read/write for each operation
# - operations: operations to execute READ|WRITE|READ_WRITE
    'IOPerf 100 10 READ_WRITE',
    'IOPerf  10 10 READ_WRITE',
    'IOPerf   1 10 READ_WRITE',


# RT-App workload:
# Params:
# - configration: tasks configuration to run
# - [configuration specific parameters]
#     'RTApp STAccount 6',
#     'RTApp RAMP',

]

## Experiments configuration

In [13]:
# Available test configurations

# Each conf can specify:
#
# - set:
#   Pointer to a function to setup the target before the experiment
#   This is usually user to confiure a specific CPUFreq governor
#
# - boot_img:
#   The boot.img to flash before rebooting the target to run the experiments
#   This is the filename for an image which has to be present in this folder:
boot_images_folder = '/tmp/bootimages/'
#
# - collect:
#   Define what we want to collect as a list of strings.
#   Supported values are
#    energy   - Use the my_conf's defined emeter to measure energy
#               consumption across an experiment.
#               NOTE: this requires an emeter configured in the
#                     previous my_conf
#    ftrace   - Collect an execution trace using trace-cmd
#    systrace - Collect an execution trace using Systrace/Atrace
#
# - wloads:
#   Defined the workloads to execute
#   If not defined all the workloads specified in test_wloads will be executed
#   Otherwise, a workload in test_workloads is executed only if it matches
#   at least one of the strings defined in this list, e.g.
#     'wloads' : ['Jankbench list', 'IOPerf']
#   will execute only workloads which name constains at least one of the two
#   specified strings

confs = {

    # SchedFreq
    'sfreq' : {
        'boot_img' : 'boot_sfreq.img',
        'set'      : set_sched,
        'wloads'   : ['IOPerf  10 '],
        'collect'  : 'energy',
    },

    # SchedUtil PELT
    'sutil_pelt' : {
        'boot_img' : 'boot_sutil_pelt.img',
        'set'      : set_schedutil,
        'wloads'   : ['IOPerf  10 '],
        'collect'  : 'energy',
    },

    # SchedUtil WALT
    'sutil_walt' : {
        'boot_img' : 'boot_sutil_walt.img',
        'set'      : set_schedutil,
        'wloads'   : ['UiBench 1', 'IOPerf  10 '],
        'collect'  : 'energy',
    },
    # Android LSK + EAS
    'eas' : {
        #'boot_img' : 'boot_eas.img',
        'set'      : set_sched,
        'wloads'   : ['UiBench 1', 'IOPerf  10'],
        'collect'  : 'energy ftrace',
    },
    'eas_systrace' : {
        'set'      : set_sched,
        'wloads'   : ['UiBench 1', 'IOPerf 10'],
        'collect'  : 'energy systrace'
    }
}

# Main

## Target Connection

In [14]:
# Cleanup Caiman energy meter temporary folders
!rm -rf /tmp/eprobe-caiman-*
# Ensure there are not other "caiman" instanced running for the specified device
# my_conf['emeter']['conf']['device_entry']
!killall caiman

caiman: no process found


In [15]:
# Decongestion ADB buffers?!?
# !for i in $(seq 100); do adb shell "ls -la"; done >/dev/null

In [16]:
# Initialize a test environment using:
te = TestEnv(my_conf, wipe=False, force_new=True)
target = te.target

2016-09-28 17:55:39,965 INFO    :         Target - Using base path: /vagrant
2016-09-28 17:55:39,967 INFO    :         Target - Loading custom (inline) target configuration
2016-09-28 17:55:39,968 INFO    :         Target - External tools using:
2016-09-28 17:55:39,970 INFO    :         Target -    ANDROID_HOME: /home/vagrant/lisa/tools/android-sdk-linux/
2016-09-28 17:55:39,972 INFO    :         Target -    CATAPULT_HOME: /home/vagrant/lisa/tools/catapult
2016-09-28 17:55:39,975 INFO    :       Platform - Loading board:
2016-09-28 17:55:39,976 INFO    :       Platform -    /vagrant/libs/utils/platforms/hikey.json
2016-09-28 17:55:39,980 INFO    :         Target - Devlib modules to load: [u'bl', u'cpufreq']
2016-09-28 17:55:39,982 INFO    :         Target - Connecting Android target [2185A639000468F1]
2016-09-28 17:55:39,984 INFO    :         Target - Connection settings:
2016-09-28 17:55:39,988 INFO    :         Target -    {'device': '2185A639000468F1'}
2016-09-28 17:55:41,395 INFO  

## Workloads Execution and Data Collection

In [17]:
experiments = ['eas']

In [18]:
# run_experiments(experiments, verbose=False, pretend=True)
results = run_experiments(experiments)

2016-09-28 17:55:54,163 WARNING :       Workload - Package [com.android.benchmark] not installed
2016-09-28 17:55:54,164 WARNING :       Workload - Workload [Jankbench] disabled
2016-09-28 17:55:58,124 WARNING :       Workload - Package [com.google.android.youtube] not installed
2016-09-28 17:55:58,126 WARNING :       Workload - Workload [YouTube] disabled
2016-09-28 17:55:58,127 INFO    :       Workload - Workloads available on target:
2016-09-28 17:55:58,128 INFO    :       Workload -   ['UiBench']
2016-09-28 17:55:59,905 INFO    : 


2016-09-28 17:55:59,907 INFO    : ========================
2016-09-28 17:55:59,910 INFO    : No image for configuration: eas...


adbd is already running as root
/system/bin/sh: can't create /sys/class/power_supply/battery/charging_enabled: No such file or directory


2016-09-28 17:56:00,350 INFO    : Running with kernel:
2016-09-28 17:56:00,353 INFO    :    Linux localhost 4.4.17-230074-gabcaffc #1 SMP PREEMPT Sun Sep 25 00:06:21 BST 2016 aarch64

2016-09-28 17:56:02,139 INFO    : Enabling collection of: energy ftrace
2016-09-28 17:56:02,141 INFO    : ------------------------
2016-09-28 17:56:02,143 INFO    : Test 1: UIBENCH in EAS configuration
2016-09-28 17:56:02,146 INFO    :         UiBench 1 TrivialAnimation 10
2016-09-28 17:56:02,147 INFO    :       Workload - Workloads available on target:
2016-09-28 17:56:02,148 INFO    :       Workload -   ['UiBench']
2016-09-28 17:56:02,157 INFO    : Set CPUFreq: SchedFreq
2016-09-28 17:56:05,865 INFO    : Set brightness: 100%
2016-09-28 17:56:07,612 INFO    : Dim screen mode: OFF
2016-09-28 17:56:09,357 INFO    : Screen timeout: 36000 [s]
2016-09-28 17:56:28,104 INFO    : Force manual orientation
2016-09-28 17:56:28,107 INFO    : Set orientation: PORTRAIT
2016-09-28 17:56:32,962 INFO    : adb -s 2185A639

In [19]:
results

{'eas': {'IOPerf  10 10 READ_WRITE': {'db_file': '/vagrant/results/Android_WorkloadsExecution/eas/IOPerf_10/data.csv',
   'dir': '/vagrant/results/Android_WorkloadsExecution/eas/IOPerf_10',
   'nrg_report': EnergyReport(channels={'Device1': 4926.1, 'Device0': 0.0}, report_file='/vagrant/results/Android_WorkloadsExecution/eas/IOPerf_10/energy.json'),
   'trace_file': '/vagrant/results/Android_WorkloadsExecution/eas/IOPerf_10/trace.dat'},
  'UiBench 1 TrivialAnimation 10': {'db_file': '/vagrant/results/Android_WorkloadsExecution/eas/UiBench_TrivialAnimation/framestats.txt',
   'dir': '/vagrant/results/Android_WorkloadsExecution/eas/UiBench_TrivialAnimation',
   'nrg_report': EnergyReport(channels={'Device1': 2461.4, 'Device0': 0.0}, report_file='/vagrant/results/Android_WorkloadsExecution/eas/UiBench_TrivialAnimation/energy.json'),
   'trace_file': '/vagrant/results/Android_WorkloadsExecution/eas/UiBench_TrivialAnimation/trace.dat'}}}

## Energy Measurements Report

In [20]:
for conf_name in experiments:
    for idx, wload_name in enumerate(wloads):
        nrg = 'NaN'
        if results[conf_name].has_key(wload_name):
            if results[conf_name][wload_name]['nrg_report']:
                nrg = results[conf_name][wload_name]['nrg_report'].channels['Device1']
        print "Energy consumption {}, {:52}: {}".format(conf_name.upper(), wload_name, nrg)

Energy consumption EAS, UiBench 1 TrivialAnimation 10                       : 2461.4
Energy consumption EAS, IOPerf 100 10 READ_WRITE                            : NaN
Energy consumption EAS, IOPerf  10 10 READ_WRITE                            : 4926.1
Energy consumption EAS, IOPerf   1 10 READ_WRITE                            : NaN
